In [3]:
'''
Author: Andre Telfer
Email: telfer006@gmail.com
'''

import cv2
import os
import numpy as np
import math
import logging
import sys
from pathlib import Path
import attr

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

@attr.dataclass
class DarknetModel:
    config_path : str = None
    weight_path : str = None
    meta_path : str = None

    def __attrs_post_init__(self):
        import darknet
        self.net = darknet.load_net_custom(self.config_path.encode("ascii"), self.weight_path.encode("ascii"), 0, 1)
        self.meta = darknet.load_meta(self.meta_path.encode("ascii"))
        
    def predict(self, frame):
        im, arr = darknet.array_to_image(frame)
        res = darknet.detect_image(self.net, self.meta, im, thresh=.5, hier_thresh=.5, nms=.45, debug= False)
        return res
        
def sample_frames_from_video(video, n_frames, output_dir, output_prefix='frame', output_ext='png'):
    '''
    Uniform sample frames from video
    @param path/str video: the video to read from
    @param int n_frames: the number of frames to collect from each video
    @param path/str output_dir: the directory to put the frames in
    @param str output_prefix: the prefix for the saved frame images
    @param str output_ext: the extension for the saved frame images
    '''
    
    if not os.path.exists(video):
        logging.critical(f'Video does not exist: {video}')
        return
    
    if type(output_dir) != Path:
        output_dir = Path(output_dir)
        
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    logging.info(f'Processing {video}')
    video_head, video_name = os.path.split(video)
    cap = cv2.VideoCapture(str(video))
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    selected_indices = np.random.choice(np.arange(frames+1), n_frames).astype(np.int64)
    selected_indices = list(sorted(selected_indices))
    logging.info(f'Selected frames: {selected_indices}')
    for index in selected_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        
        if not ret:
            logging.error(f"Error fetching frame {index}, skipping")
            continue
        
        # Pad the frame index, this is the standard format for DLC extracted images
        padded_index = str(int(index)).zfill(math.ceil(np.log(frames)/np.log(10)))
        
        output_filepath = output_dir / f"{output_prefix}{padded_index}.{output_ext}"
        cv2.imwrite(str(output_filepath), frame)
        logging.info(f"Wrote frame {index} to {output_filepath}")

def sample_frames_from_videos(videos, n_frames, output_dir, output_prefix='frame', output_ext='png', flat=False):
    '''
    Uniform sample frames from video
    @param path/str video: the videos to read from
    @param int n_frames: the number of frames to collect from each video
    @param path/str output_dir: the directory to put the frames in
    @param str output_prefix: the prefix for the saved frame images
    @param str output_ext: the extension for the saved frame images
    @param bool flat: if flat, then all frames are saved to the same folder and given an output prefix based on their video
    '''
    
    if type(output_dir) != Path:
        output_dir = Path(output_dir)
    
    for video in videos:
        video_output_dir = output_dir
        video_output_prefix = output_prefix
        
        head, tail = os.path.split(video)
        name, ext =  os.path.splitext(tail)
        if not flat:
            video_output_dir /= name
        else:
            video_output_prefix = f'{name}_{output_prefix}'
            
        sample_frames_from_video(
            video, 
            n_frames, 
            output_prefix=video_output_prefix, 
            output_ext=output_ext, 
            output_dir=video_output_dir
        )

# Example Usage

## Example file directory

In [ ]:
video_dir = Path('/workspace/gs/rat-emotion/24fps') 
videos = [video_dir / video for video in os.listdir(video_dir)]

sample_frames_from_videos(videos, 10, '/tmp/test', flat=True)

## Example with glob pattern

In [26]:
import glob

output_dir = '/storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02'
videos = glob.glob('/storage/gs/rat-emotion/videos/24fps/rat[2-6]-lps*')
sample_frames_from_videos(videos, 3, output_dir=output_dir, flat=True)

videos = glob.glob('/storage/gs/rat-emotion/videos/24fps/rat[2-6]-control*')
sample_frames_from_videos(videos, 7, output_dir=output_dir, flat=True)

2020-06-02 19:33:27,400 | INFO : Processing /storage/gs/rat-emotion/videos/24fps/rat5-lps1.mp4
2020-06-02 19:33:27,415 | INFO : Selected frames: [1682, 2262, 7355]
2020-06-02 19:33:28,740 | INFO : Wrote frame 1682 to /storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02/rat5-lps1_frame01682.png
2020-06-02 19:33:30,397 | INFO : Wrote frame 2262 to /storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02/rat5-lps1_frame02262.png
2020-06-02 19:33:31,207 | INFO : Wrote frame 7355 to /storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02/rat5-lps1_frame07355.png
2020-06-02 19:33:31,216 | INFO : Processing /storage/gs/rat-emotion/videos/24fps/rat3-lps1.mp4
2020-06-02 19:33:31,223 | INFO : Selected frames: [437, 2791, 3425]
2020-06-02 19:33:32,488 | INFO : Wrote frame 437 to /storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02/rat3-lps1_frame00437.png
2020-06-02 19:33:33,028 | INFO : Wrote frame 2791 to /storage/gs/rat-emotion/darknet/unlabeled_frames/2020-06-02/rat3-lps1

## Example with Darknet

In [4]:
import glob

DarknetModel(
    config_path = "/storage/gs/rat-emotion/darknet/cfg/2020-06-02_yolo-obj.cfg",
    weight_path = "/storage/gs/rat-emotion/darknet/weights/2020-06-07/2020-06-02_yolo-obj_7000.weights",
    meta_path = "/storage/gs/rat-emotion/darknet/labeled_data/2020-06-07/obj.data"
)


ModuleNotFoundError: No module named 'darknet'